In [14]:
import json
from datetime import date, timedelta

import pandas as pd
import matplotlib.pyplot as plt
from langdetect import detect
import glob
import os

In [15]:
from searchtweets import ResultStream, gen_rule_payload, load_credentials, collect_results

In [16]:
premium_search_args = load_credentials("creds_twitter_keys_archive.yaml",
                                       yaml_key="search_tweets_premium",
                                       env_overwrite=False)

Grabbing bearer token from OAUTH


In [17]:
def get_tweets(search_words, start_date, end_date, num_twt):
    rule = gen_rule_payload(search_words,
                        from_date=start_date.strftime("%Y-%m-%d"),
                        to_date=end_date.strftime("%Y-%m-%d"),
                        results_per_call=num_twt)
    
    tweets = collect_results(rule, max_results=num_twt, result_stream_args=premium_search_args)
    
    tweets_col_filter = [[tweet['created_at'], tweet['text'], tweet['id'], tweet['user']['id'], 
                tweet['user']['location'], tweet['user']['followers_count'], tweet['user']['lang'], 
                tweet['user']['time_zone'], tweet['retweet_count'], tweet['favorite_count']
               ] 
               for tweet in tweets]
    
    tweet_df = pd.DataFrame(data=tweets_col_filter, 
                    columns=['created_at', 'text', 'id', 'user_id', 'user_loc', 'user_followers', 'user_lang',
                            'user_time_zone', 'retweet_count', 'fav_count'])
    
    
    return tweet_df

In [18]:
def save_tweets(tweet_df, start_date, search_key):
    timestampStr = start_date.strftime("%Y_%m_%d")
    
    tweet_df.to_csv('data2/raw/candidate/2012/' + search_key + '/' + search_key + '_' + timestampStr + '.csv')

In [19]:
def get_and_save_twts_by_day(start_date, search_str, num_days):
    
    for d in range(num_days):
        end_date = start_date + timedelta(days=1)
        #search_str = 'trump OR biden'
        num_twt = 500

        tweet_df = get_tweets(search_str, start_date, end_date, num_twt)
        save_tweets(tweet_df, start_date, search_str)

        print(start_date.strftime("%Y-%m-%d"), end_date.strftime("%Y-%m-%d"))

        start_date = start_date + timedelta(days=1)
    

In [21]:
get_and_save_twts_by_day(date(2012, 10, 30), 'romney', 1)
#hillary

2012-10-30 2012-10-31
